In [1]:
import pandas as pd
import re
import datetime

In [2]:
import pandas as pd

df = pd.read_csv('D:\\KHDL\\project2,3,4\\info_cars 1520_1999.csv',usecols=list(range(9))+[18])
df.head(10)

,name,sell_time,produce_year,current_status,traveled_kilometer,origin,shape,gear_box,engine,owner_address
0,Xe Hyundai Tucson 1.6 AT Turbo HTRAC Đặc biệt ...,Đăng ngày 4/10/2025 . Xem 12 lượt,2023,Xe đã dùng,"21,000 Km",Lắp ráp trong nước,Crossover,Số tự động,Xăng 1.6 L,"Địa chỉ: 14/1 Đại lộ Bình Dương, khu phố Bình ..."
1,Xe Lexus RX 350 2021 - 3 Tỷ 680 Triệu,Đăng ngày 5/10/2025 . Xem 11 lượt,2021,Xe đã dùng,"10,000 Km",Nhập khẩu,SUV,Số tự động,Xăng 3.5 L,"Địa chỉ: 70 Trương Công Giai, Phường Dịch Vọng..."
2,Xe BMW X4 xDrive20i M Sport 2020 - 1 Tỷ 780 Triệu,Đăng ngày 4/10/2025 . Xem 10 lượt,2020,Xe đã dùng,0 Km,Nhập khẩu,SUV,Số tự động,Xăng 2.0 L,"Địa chỉ: 17 HDi Mạc Thái Tông , Trung Hoà , Cầ..."
3,Xe Porsche Macan 2.0 2021 - 3 Tỷ 90 Triệu,Đăng ngày 6/10/2025 . Xem 7 lượt,2021,Xe đã dùng,"45,000 Km",Nhập khẩu,SUV,Số tự động,Xăng 2.0 L,"Địa chỉ: 66 - 68 Đường số 7, P. Tân Kiểng, Quậ..."
4,Xe Hyundai Tucson 2.0 AT Đặc biệt 2025 - 819 T...,Đăng ngày 5/10/2025 . Xem 15 lượt,2025,Xe mới,NaN,Lắp ráp trong nước,Crossover,Số tự động,Xăng 2.0 L,"Địa chỉ: Do Lộ, Phường Yên Nghĩa, Quận Hà Đông..."
5,Xe Lexus RX 300 2021 - 2 Tỷ 779 Triệu,Đăng ngày 6/10/2025 . Xem 9 lượt,2021,Xe đã dùng,"40,000 Km",Nhập khẩu,SUV,Số tự động,Xăng 2.0 L,"Địa chỉ: Toà CT2 Chung cư văn phòng Quốc Hội, ..."
6,Xe Hyundai SantaFe Tiêu chuẩn 2.5L 2022 - 888 ...,Đăng ngày 5/10/2025 . Xem 6 lượt,2022,Xe đã dùng,"59,000 Km",Lắp ráp trong nước,SUV,Số tự động,Xăng 2.5 L,"Địa chỉ: 26 Đường Cầu Diễn, Phú Diễn, Bắc Từ L..."
7,Xe Lexus RX 300 2021 - 2 Tỷ 880 Triệu,Đăng ngày 5/10/2025 . Xem 6 lượt,2021,Xe đã dùng,"35,000 Km",Nhập khẩu,SUV,Số tự động,Xăng 2.0 L,"Địa chỉ: 70 Trương Công Giai, Phường Dịch Vọng..."
8,Xe Suzuki Ciaz 1.4 AT 2019 - 345 Triệu,Đăng ngày 6/10/2025 . Xem 6 lượt,2019,Xe đã dùng,"72,000 Km",Nhập khẩu,Sedan,Số tự động,Xăng 1.4 L,"Địa chỉ: 202/2 Đt 743, P. An Phú, Tp Thuận An ..."
9,Xe Mitsubishi Xforce Ultimate 2025 - 610 Triệu,Đăng ngày 6/10/2025 . Xem 7 lượt,2025,Xe mới,NaN,Nhập khẩu,SUV,Số tự động,Xăng 1.5 L,"Địa chỉ: Số 1 Nguyễn Văn Linh , Long Biên Hà Nội"


In [3]:
def split_name_and_price(s):
    if pd.isna(s):
        return pd.Series([None, None])
    s_str = str(s)
    if '-' in s_str:
        parts = s_str.rsplit('-', 1)
        if len(parts) == 2:
            return pd.Series([parts[0].strip(), parts[1].strip()])
    return pd.Series([s_str.strip(), None])

def normalize_price(price_str):
    """Chuyển đổi giá về đơn vị triệu VNĐ"""
    if pd.isna(price_str):
        return None
    price_str = str(price_str).strip()
    # Tìm tất cả các số trong chuỗi
    numbers = re.findall(r'\d+', price_str)
    if not numbers:
        return None
    
    # Chuyển đổi dựa trên đơn vị
    if 'Tỷ' in price_str or 'tỷ' in price_str:
        ty = int(numbers[0]) * 1000  # 1 tỷ = 1000 triệu
        trieu = int(numbers[1]) if len(numbers) > 1 else 0
        return ty + trieu
    elif 'Triệu' in price_str or 'triệu' in price_str:
        return int(numbers[0])
    else:
        return int(numbers[0])
    
def extract_daytime(s):
    if pd.isna(s):
        return None
    match = re.search(r'(\d{1,2})/(\d{1,2})/(\d{4})', s)
    if match:
        day, month, year = map(int, match.groups())
        date_obj = datetime.datetime(year,month,day)
        return str(date_obj.strftime(f"%d/%m/%Y"))
    return None

def parse_km(s):
    if pd.isna(s):
        return None
    if not isinstance(s, str):
        return None
    # Xóa mọi ký tự không phải số
    digits = re.sub(r'[^\d]', '', s)
    return int(digits) if digits else None

In [4]:
df = pd.read_csv('D:\\KHDL\\project2,3,4\\info_cars 1520_1999.csv',usecols=list(range(9))+[18])
dataframe = df

# DEBUG: Xem dữ liệu cột đầu tiên
# print("=== 5 dòng đầu của cột 0 ===")
# print(dataframe.iloc[:5, 0])
# print("\n")

# Áp dụng hàm lên CỘT ĐẦU TIÊN (index 0)
dataframe[['name','price']] = dataframe.iloc[:, 0].apply(split_name_and_price)

# Áp dụng cho cột price
dataframe['price'] = dataframe['price'].apply(normalize_price)

# Lấy ngày ra khỏi cột sell_time
dataframe['sell_time'] = dataframe['sell_time'].apply(extract_daytime)

# Áp dụng lấy số km đã đi
dataframe['traveled_kilometer'] = dataframe['traveled_kilometer'].apply(parse_km)

# print(dataframe[['name','sell_time','price','traveled_kilometer']].head(50))
dataframe.to_csv('ouput.csv',index=False)